In [24]:
import pandas as pd
import os
from src.prompts import language_dict

def get_keys_by_value(d, target_value):
    return [key for key, value in d.items() if value == target_value][0]

languages = ["Afrikaans", "Arabic", "Chinese_(Simplified)", "English", "French", "German", "Hebrew", "Indonesian", "Italian", "Japanese", "Korean", "Spanish", "Turkish", "Vietnamese"]
models = [
    "jwhj/Qwen2.5-Math-1.5B-OREO", "nvidia/AceMath-1.5B-Instruct",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", "Qwen/Qwen2.5-Math-1.5B-Instruct",
    "PRIME-RL/Eurus-2-7B-PRIME", "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    "nvidia/AceMath-7B-Instruct", "Qwen/Qwen2.5-Math-7B-Instruct",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B", "o3-mini"
]
iso = [get_keys_by_value(language_dict, l) for l in languages]

res = {"model": models}
for i, l in enumerate(languages):
    # for task in ["math500", "math100"]:
    #     df = pd.read_csv(os.path.join("score_result", f"{task}.csv"))
    #     res[f"{l}_{task[-3:]}"] = list(df[l])
    df_500, df_100 = pd.read_csv(os.path.join("score_result", "math500.csv")), pd.read_csv(os.path.join("score_result", "math100.csv"))
    res[iso[i]] = []
    for m in models:
        m = m.replace("/", "_")
        df_500_index, df_100_index = list(df_500["model"]).index(m), list(df_100["model"]).index(m)
        # res[iso[i]].append(abs(list(df_500[l])[df_500_index] - list(df_100[l])[df_100_index]))
        res[iso[i]].append(list(df_500[l])[df_500_index])
res = pd.DataFrame(res)
res.to_excel("check.xlsx", index=False)
# .to_csv(f"result_{task}.csv", index=False)

In [ ]:
import pandas as pd
import random
import os
import numpy as np
from tqdm.auto import tqdm

result_dict = {
    "math500": {
        "jwhj/Qwen2.5-Math-1.5B-OREO": [],
        "nvidia/AceMath-1.5B-Instruct": [],
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B": [],
        "Qwen/Qwen2.5-Math-1.5B-Instruct": [],
        "PRIME-RL/Eurus-2-7B-PRIME": [],
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B": [],
        "nvidia/AceMath-7B-Instruct": [],
        "Qwen/Qwen2.5-Math-7B-Instruct": [],
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B": [],
        "o3-mini": []
    },
    "math100": {
        "jwhj/Qwen2.5-Math-1.5B-OREO": [],
        "nvidia/AceMath-1.5B-Instruct": [],
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B": [],
        "Qwen/Qwen2.5-Math-1.5B-Instruct": [],
        "PRIME-RL/Eurus-2-7B-PRIME": [],
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B": [],
        "nvidia/AceMath-7B-Instruct": [],
        "Qwen/Qwen2.5-Math-7B-Instruct": [],
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B": [],
        "o3-mini": []
    }
}
model_list = list(result_dict["math500"].keys())

n, m = 50, 100

for _ in tqdm(range(m)):
    random_index_500, random_index_100 = random.sample(list(range(499)), n), random.sample(list(range(99)), n)
    for model in model_list:
        for task in ["math500", "math100"]:
            check = []
            for file in [f for f in os.listdir(os.path.join("score_logs", task, model.replace("/", "_"))) if ".jsonl" in f]:
                df = pd.read_csv(os.path.join("score_logs", task, model.replace("/", "_"), file))
                if task == "math500":
                    df = df[df.index.isin(random_index_500)]
                elif task == "math100":
                    df = df[df.index.isin(random_index_100)]
                df.reset_index(inplace=True, drop=True)
                check.append(list(df["correctness"]).count(True) / len(df) * 100)
            result_dict[task][model].append(np.mean(check))

import pickle
with open('boot.pickle', "wb") as f:
    pickle.dump(result_dict, f)

  0%|          | 0/100 [00:00<?, ?it/s]

TypeError: dump() missing required argument 'file' (pos 2)

In [47]:
import scipy.stats
import pickle

with open("boot.pickle", "rb") as f:
    a = pickle.load(f)

print(f"--------------------- Evaluation Score T-test ---------------------")
check_500, check_100 = [], []
for key in a["math500"].keys():
    # tau, p_value = scipy.stats.kendalltau(a["math500"][key], a["math100"][key])  # Welch's t-test 적용
    check_500.append(np.mean(a["math500"][key]))
    check_100.append(np.mean(a["math100"][key]))
    # print(f"{key} - Kendall-Tau: {tau:.4f}, P-Value: {p_value:.4f}")
scipy.stats.kendalltau(check_500, check_100)

--------------------- Evaluation Score T-test ---------------------


SignificanceResult(statistic=0.9555555555555554, pvalue=5.5114638447971785e-06)